# Bibliotecas

## Instalações

In [9]:

#!pip install -U scikit-learn
#!pip install seaborn
#!pip install nltk
!pip install wordcloud 

## Importações

In [38]:
import nltk
from nltk import tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import silhouette_samples, silhouette_score, v_measure_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans


# Análise Exploratória

In [2]:
licitacoes = pd.read_csv('C:\Gabriel\Pos\Classificação Licitômetro\TodasLicitacoes.csv',sep=';')

In [6]:
licitacoes.head(20)

,DsObjetoLicitacao
0,
1,...
2,Seleção e contratação de empresa de en...
3,ESTABILIZADOR DE TENSÃO 2.OOO VA
4,Para fornecer água para o setor de educação...
5,Para prestar serviços na paresentação do gr...
6,PRESTAÇÃO DE SERVIÇO NO FORNECIMENTO DE 14 ...
7,Serviço de reprografia preto e branco até a...
8,Solicitação para aquisição de materiais de ...
9,Solicitação para serviço de instalação dos ...


In [4]:
print(len(licitacoes))

1515582


# Tratamento de Dados

## Removendo StopWords

In [76]:
palavras_irrelevantes = nltk.corpus.stopwords.words("portuguese")
frase_processada = list()
token_espaco = tokenize.WhitespaceTokenizer()

for opiniao in licitacoes["DsObjetoLicitacao"]:
    nova_frase = list()
    palavras_texto = token_espaco.tokenize(opiniao)
    for palavra in palavras_texto:
        if palavra not in palavras_irrelevantes:
            nova_frase.append(palavra)
    frase_processada.append(' '.join(nova_frase))
    
licitacoes["retirada_stop_words"] = frase_processada

In [77]:
licitacoes.sample(10)

,DsObjetoLicitacao,retirada_stop_words
1354589,Locação de imóvel localizado no Povoado de Ilh...,Locação imóvel localizado Povoado Ilha Camaleã...
1194788,"AQUISIÇÃO DE QUATRO CAIXA DE ISOPOR 165 LTS, P...","AQUISIÇÃO DE QUATRO CAIXA DE ISOPOR 165 LTS, P..."
241307,confeccao de portas,confeccao portas
1425426,Prestação de serviços na locação de uma CAÇAMB...,Prestação serviços locação CAÇAMBA placa polic...
1506310,VALOR EMPENHADO PARA ATENDER A DESPESA COM AQU...,VALOR EMPENHADO PARA ATENDER A DESPESA COM AQU...
459364,Contratação do Show Artístico de “Caviar com R...,Contratação Show Artístico “Caviar Rapadura” a...
660871,LOCAÇÃO DE IMÓVEL SITUADO NA RUA SÃO PAULO N. ...,LOCAÇÃO DE IMÓVEL SITUADO NA RUA SÃO PAULO N. ...
1393333,PRESTAÇÃO DE SERVIÇO COMO SERRALHEIRO EM RECUP...,PRESTAÇÃO DE SERVIÇO COMO SERRALHEIRO EM RECUP...
476169,AQUISIÇAO DE 1000 (HUM MIL) CÁPSULAS PARA CHIB...,AQUISIÇAO DE 1000 (HUM MIL) CÁPSULAS PARA CHIB...
1338639,FORNECIMENTO DE LANCHES PARA OS VACINADORES DA...,FORNECIMENTO DE LANCHES PARA OS VACINADORES DA...


In [50]:
def gerando_bag_of_words_vetorizer(dataset,nome_coluna):
    vectorizer = TfidfVectorizer(max_df=0.5, max_features=50, min_df=2, stop_words='english', use_idf=True)
    bag_of_words = vectorizer.fit_transform(dataset.DsObjetoLicitacao)
  
    
    return vetorizar,bag_of_words

In [63]:
vetorizar,bag_of_words = gerando_bag_of_words_vetorizer(licitacoes,"DsObjetoLicitacao")



pd.DataFrame.sparse.from_spmatrix(
            bag_of_words,
            columns = vetorizar.get_feature_names_out()
       
    )

,01,AO,AQUISIÇÃO,AS,ATENDER,Aquisição,BA,COM,CONTRATAÇÃO,Contratação,...,deste,do,dos,em,empresa,município,na,no,para,serviços
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1515577,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1515578,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1515579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1515580,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
print(vetorizar.shape)

(1515582, 50)


In [67]:

random_state =42
for num_cluster in range(2,30):
    kmeans = KMeans(n_clusters=num_cluster, init='k-means++', max_iter=100, n_init=1)
    kmeans.fit(bag_of_words)

kmeans.predict(bag_of_words)

array([ 1, 22,  8, ...,  1, 15,  3])

In [68]:
# Exibir os centros dos clusters e as palavras mais importantes em cada cluster
print("Top terms per cluster:")
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]


Top terms per cluster:


In [70]:
vetorizer

<1515582x50 sparse matrix of type '<class 'numpy.int64'>'
	with 9079959 stored elements in Compressed Sparse Row format>

In [72]:
terms = vetorizar.get_feature_names_out()
num_clusters = 10
for i in range(num_clusters):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print()

# Atribuir rótulos aos clusters
labels = kmeans.labels_

Cluster 0:
 NA
 Município
 DE
 SE
 COM
 serviços
 Aquisição
 SAÚDE
 QUE
 AO

Cluster 1:
 AO
 Municipal
 01
 SE
 deste
 EMPRESA
 QUE
 de
 para
 serviços

Cluster 2:
 na
 DE
 da
 SE
 QUE
 CONTRATAÇÃO
 MUNICIPAL
 em
 SAÚDE
 DAS

Cluster 3:
 DE
 QUE
 no
 SE
 EMPRESA
 AS
 BA
 em
 DESTE
 SAÚDE

Cluster 4:
 AS
 DE
 QUE
 SAÚDE
 em
 AO
 EM
 SE
 EMPRESA
 DESTE

Cluster 5:
 MUNICIPIO
 DAS
 Municipal
 DE
 no
 SAÚDE
 da
 QUE
 Aquisição
 MUNICIPAL

Cluster 6:
 MUNICÍPIO
 MUNICIPIO
 DAS
 da
 no
 SE
 MUNICIPAL
 DE
 SAÚDE
 EMPRESA

Cluster 7:
 CONTRATAÇÃO
 DE
 no
 AS
 SE
 ATENDER
 QUE
 EMPRESA
 DAS
 MUNICIPAL

Cluster 8:
 MUNICIPAL
 CONTRATAÇÃO
 DE
 no
 AS
 SE
 QUE
 AO
 SAÚDE
 EMPRESA

Cluster 9:
 SAÚDE
 DOS
 SE
 empresa
 no
 da
 DE
 AS
 MUNICIPAL
 CONTRATAÇÃO

